In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install neuralprophet
!pip install git+https://github.com/ourownstory/neural_prophet.git numpy==1.23.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 19.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x

In [ ]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
import matplotlib.pyplot as plt
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error



ERROR:NP.plotly:Importing plotly failed. Interactive plots will not work.
ERROR:NP.plotly:Importing plotly failed. Interactive plots will not work.


# Load historical load and temperature

In [ ]:
def read_ncep_data(filename):
    _df = pd.read_csv(filename,sep=";", skiprows=26, names=col_names)
    _df['Time'] = _df['Time'].replace("24:00", "00:00")

    # Concatenate Date and Time columns
    _df['Datetime'] = _df['Date'] + " " + _df['Time']

    # Convert to datetime type
    _df['Datetime'] = pd.to_datetime(_df['Datetime'])

    # Identify rows where 'Time' is "00:00" and adjust the 'Date' by adding 1 day

    next_day_mask = _df['Time'] == "00:00"
    _df.loc[next_day_mask, 'Datetime'] += pd.DateOffset(days=1)
    _df['Datetime'] = _df['Datetime'] + pd.DateOffset(hours=7)

    dropped_cols = ['Date', 'Time']
    _df.drop(columns=dropped_cols, inplace=True)
    _df.set_index('Datetime', inplace=True)
    return _df

In [ ]:
# read NCEP data
col_names = ['Date', 'Time','Tncep', 'RH', 'Pressure', 'WS', 'WD', 'rain_fall', 'snow_fall', 'snow_depth', 'Incep']


ncep_df_2 = read_ncep_data('/content/drive/MyDrive/data for senior prj/siteCU04_2023-01-01to2023-12-31.csv') #import 2023 data
ncep_df_2['Tncep'] = ncep_df_2['Tncep']-273.5
ncep_df_2['Incep'] = ncep_df_2['Incep'] * 4
ncep_df =ncep_df_2.reset_index()
ncep_df

,Datetime,Tncep,RH,Pressure,WS,WD,rain_fall,snow_fall,snow_depth,Incep
0,2023-01-01 07:15:00,21.01,59.42,1015.53,1.12,348.45,0.0,0.0,0.0,49.2732
1,2023-01-01 07:30:00,21.35,58.39,1015.66,1.23,352.57,0.0,0.0,0.0,89.5056
2,2023-01-01 07:45:00,21.69,57.36,1015.79,1.34,356.02,0.0,0.0,0.0,135.5892
3,2023-01-01 08:00:00,22.04,56.33,1015.92,1.46,358.93,0.0,0.0,0.0,187.1732
4,2023-01-01 08:15:00,22.38,55.30,1016.05,1.58,1.40,0.0,0.0,0.0,243.8560
...,...,...,...,...,...,...,...,...,...,...
35035,2024-01-01 06:00:00,25.17,60.25,1010.75,2.89,332.32,0.0,0.0,0.0,0.0000
35036,2024-01-01 06:15:00,25.11,60.64,1010.83,2.86,333.28,0.0,0.0,0.0,0.0000
35037,2024-01-01 06:30:00,25.06,61.02,1010.91,2.84,334.26,0.0,0.0,0.0,0.0000
35038,2024-01-01 06:45:00,25.00,61.41,1011.00,2.82,335.25,0.0,0.0,0.0,0.0028


In [ ]:
load_df = pd.read_csv('/content/drive/MyDrive/data for senior prj/load_data_15minresample_032023_022024.csv', parse_dates=['datetime'],usecols=['datetime','Ptot (kW)'] )
# use only Temp and Incep
ncep_df = ncep_df[['Datetime','Tncep','Incep']]
ncep_df = ncep_df.rename(columns = {'Datetime' : 'datetime'})

In [ ]:
df = pd.DataFrame()
#datetime range of load data
df['datetime'] =  pd.date_range(start = "2023-03-01", end='2023-12-31',freq='15min')
df = df.merge(ncep_df,how='left',on='datetime')
df = df.merge(load_df,how='left',on='datetime')



In [ ]:
df

,datetime,Tncep,Incep,Ptot (kW)
0,2023-03-01 00:00:00,26.99,0.0,0.843212
1,2023-03-01 00:15:00,26.89,0.0,0.910691
2,2023-03-01 00:30:00,26.79,0.0,0.902213
3,2023-03-01 00:45:00,26.68,0.0,0.907833
4,2023-03-01 01:00:00,26.58,0.0,0.820361
...,...,...,...,...
29276,2023-12-30 23:00:00,29.31,0.0,0.687662
29277,2023-12-30 23:15:00,29.18,0.0,0.741668
29278,2023-12-30 23:30:00,29.05,0.0,0.715135
29279,2023-12-30 23:45:00,28.91,0.0,0.688664


In [ ]:
# impute missing load by using same day in a week before
df.loc[(df['datetime']>= '2023-04-05') & (df['datetime'] <= '2023-04-06'),'Ptot (kW)'] = df[(df['datetime']>= '2023-03-28') & (df['datetime'] <= '2023-03-29')]['Ptot (kW)'].values
df.loc[(df['datetime']>= '2023-07-11') & (df['datetime'] <= '2023-07-13'),'Ptot (kW)'] = df[(df['datetime']>= '2023-07-04') & (df['datetime'] <= '2023-07-06')]['Ptot (kW)'].values

In [ ]:
df

,datetime,Tncep,Incep,Ptot (kW)
0,2023-03-01 00:00:00,26.99,0.0,0.843212
1,2023-03-01 00:15:00,26.89,0.0,0.910691
2,2023-03-01 00:30:00,26.79,0.0,0.902213
3,2023-03-01 00:45:00,26.68,0.0,0.907833
4,2023-03-01 01:00:00,26.58,0.0,0.820361
...,...,...,...,...
29276,2023-12-30 23:00:00,29.31,0.0,0.687662
29277,2023-12-30 23:15:00,29.18,0.0,0.741668
29278,2023-12-30 23:30:00,29.05,0.0,0.715135
29279,2023-12-30 23:45:00,28.91,0.0,0.688664


In [ ]:
#check if missing load has been imputed
import plotly.graph_objects as go
import plotly.express as px
#df_ecal = df.loc[(df['datetime']>= '2023-04-05') & (df['datetime'] <= '2023-04-06')]
df_ecal = df.loc[(df['datetime']>= '2023-07-11') & (df['datetime'] <= '2023-07-13')]
fig = px.line(df_ecal, x=df_ecal['datetime'],  y=['Ptot (kW)'])
fig.show()

# Add lab machine schedule as  0 1 variable
## lab day 0 no lab 1 have lab
## lab hour using that hour as value i.e. 13:00, 13:45 -> 13 14:10,14:15 -> 14

In [ ]:
df['lab_hour'] = 0
df['lab_day'] = 0
machine_lab_day_list = ['2023-01-16', #w2
                '2023-01-23', #w3
                '2023-01-30', #w4
                '2023-02-13', #w6
                '2023-02-20', #w7
                '2023-02-27', #w8
                '2023-03-20', #w11
                '2023-03-27', #w12
                '2023-04-03', #w13
                '2023-04-24', #w16
                '2023-05-01'  #w17
                ] #only Monday of lab weeks
machine_lab_day_list = [(datetime.datetime.strptime(day,'%Y-%m-%d'),datetime.datetime.strptime(day,'%Y-%m-%d') + datetime.timedelta(days=4)) for day in machine_lab_day_list] #full lab schedule
power_lab_day_list = [ '2023-01-20', #w2
                                  '2023-01-27', #w3
                                  '2023-02-10',#w5
                                  '2023-02-17',#w6
                                  '2023-02-24',#w7
                                  '2023-03-17',#w10
                                  '2023-03-24',#w11
                                  '2023-03-31',#w12
                                  '2023-04-07',#w13
                                  '2023-04-21' #w15
                        ] #Power lab is only on Friday
power_lab_day_list = [(datetime.datetime.strptime(day,'%Y-%m-%d'),datetime.datetime.strptime(day,'%Y-%m-%d') + datetime.timedelta(days=1)) for day in power_lab_day_list]
for start,end in machine_lab_day_list :
    df.loc[(df['datetime'] >= start) & (df['datetime'] <= end),'lab_day' ] = 1
    for i in range(4) :
        #assume lab preparation time is @ 13:00 and lab is ending @ 17:00
        df.loc[(df['datetime'] >= start + datetime.timedelta(days=i,hours=13))
               & (df['datetime'] <= start + datetime.timedelta(days=i,hours=17) ),'lab_hour' ] = df.loc[(df['datetime'] >= start + datetime.timedelta(days=i,hours=13))
                                                                                                                                            & (df['datetime'] <= start + datetime.timedelta(days=i,hours=17) ),'datetime' ].dt.hour
for start,end in power_lab_day_list :
    df.loc[(df['datetime'] >= start) & (df['datetime'] <= end),'lab_day' ] = 1
    df.loc[(df['datetime'] >= start + datetime.timedelta(hours=13))
               & (df['datetime'] <= start + datetime.timedelta(hours=17) ),'lab_hour' ] = df.loc[(df['datetime'] >= start + datetime.timedelta(hours=13))
                                                                                                                                          & (df['datetime'] <= start + datetime.timedelta(hours=17) ),'datetime' ].dt.hour

In [ ]:
#impute minor missing
df = df.set_index('datetime').interpolate(method='linear', axis=0)
df

,Tncep,Incep,Ptot (kW),lab_hour,lab_day
datetime,,,,,
2023-03-01 00:00:00,26.99,0.0,0.843212,0,1
2023-03-01 00:15:00,26.89,0.0,0.910691,0,1
2023-03-01 00:30:00,26.79,0.0,0.902213,0,1
2023-03-01 00:45:00,26.68,0.0,0.907833,0,1
2023-03-01 01:00:00,26.58,0.0,0.820361,0,1
...,...,...,...,...,...
2023-12-30 23:00:00,29.31,0.0,0.687662,0,0
2023-12-30 23:15:00,29.18,0.0,0.741668,0,0
2023-12-30 23:30:00,29.05,0.0,0.715135,0,0


# Train test split

In [ ]:
prophet_df = pd.DataFrame({'ds':df.index,
                           'y' : df['Ptot (kW)'],
                           'Tncep'	: df['Tncep'],
                           'Incep'	: df['Incep'],
                           'lab_hour' : df['lab_hour'],
                           'lab_day' : df['lab_day']
                           })

In [ ]:
#check if missing load has been imputed
import plotly.graph_objects as go
import plotly.express as px
#df_ecal = df.loc[(df['datetime']>= '2023-04-05') & (df['datetime'] <= '2023-04-06')]
df_ecal = prophet_df
fig = px.line(df_ecal, x=df_ecal.index,  y=['Tncep'])
fig.show()

In [ ]:
# split_idx = (prophet_df.shape[0]*1)//5
# train_data = prophet_df.iloc[split_idx:,:] # 80:20 train:test
# test_data = prophet_df.iloc[:split_idx,:]
train_data = prophet_df.iloc[:(prophet_df.shape[0]*4)//5,:] # 80:20 train:test
test_data = prophet_df.iloc[(prophet_df.shape[0]*4)//5:,:]

In [ ]:
train_data

,ds,y,Tncep,Incep,lab_hour,lab_day
datetime,,,,,,
2023-03-01 00:00:00,2023-03-01 00:00:00,0.843212,26.99,0.0,0,1
2023-03-01 00:15:00,2023-03-01 00:15:00,0.910691,26.89,0.0,0,1
2023-03-01 00:30:00,2023-03-01 00:30:00,0.902213,26.79,0.0,0,1
2023-03-01 00:45:00,2023-03-01 00:45:00,0.907833,26.68,0.0,0,1
2023-03-01 01:00:00,2023-03-01 01:00:00,0.820361,26.58,0.0,0,1
...,...,...,...,...,...,...
2023-10-30 22:45:00,2023-10-30 22:45:00,3.273279,25.38,0.0,0,0
2023-10-30 23:00:00,2023-10-30 23:00:00,2.989227,25.27,0.0,0,0
2023-10-30 23:15:00,2023-10-30 23:15:00,3.038815,25.16,0.0,0,0


In [ ]:
nnProphet_model = NeuralProphet(
    yearly_seasonality=False,  # Turn off yearly seasonality (data has only 03-2023 to 12-2023)
    weekly_seasonality=True,  # Turn on weekly seasonality (load may have weekly pattern)
    daily_seasonality=True,  # Enable daily seasonality load may have daily patterns
    n_forecasts=288, # 3 day ahead (15 min resolution)
    n_lags=96, # 1 day lag (15 min resolution)
    impute_missing=True

    )

#if future regress need temp solar predict
nnProphet_model.add_future_regressor('Tncep')
nnProphet_model.add_future_regressor('Incep')
nnProphet_model.add_future_regressor('lab_hour')
nnProphet_model.add_future_regressor('lab_day')   #change to future reg
learning_rate = 0.01  # Set an initial learning rate

# Training the model with manual learning rate adjustment
metrics = nnProphet_model.fit(train_data, freq="15T", learning_rate=learning_rate)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency 15T corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency 15T corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - 15T
INFO:NP.df_utils:Defined frequency is equal to major frequency - 15T
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO:NP.config:Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 128
INFO:NP.config:Auto-set batch_size to 128
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 40
INFO:NP.config:Auto-set epochs to 40


Training: 0it [00:00, ?it/s]

In [ ]:
# raw true based on forecast start
# default based on datetime target
forecast = nnProphet_model.predict(test_data)
#forecast = nnProphet_model.predict(prophet_df,raw=True)
forecast = forecast.dropna()
forecast

INFO - (NP.df_utils._infer_frequency) - Major frequency 15T corresponds to 99.983% of the data.
INFO:NP.df_utils:Major frequency 15T corresponds to 99.983% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - 15T
INFO:NP.df_utils:Defined frequency is equal to major frequency - 15T
INFO - (NP.df_utils._infer_frequency) - Major frequency 15T corresponds to 99.984% of the data.
INFO:NP.df_utils:Major frequency 15T corresponds to 99.984% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - 15T
INFO:NP.df_utils:Defined frequency is equal to major frequency - 15T
INFO - (NP.data.processing._handle_missing_data) - Dropped 288 rows at the end with NaNs in future regressors.
INFO:NP.data.processing:Dropped 288 rows at the end with NaNs in future regressors.


Predicting: 181it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.10/dist-packages/neuralprophet/data/process.py:95: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.10/dist-packages/neuralprophet/data/process.py:95: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-frag

,ds,y,yhat1,yhat2,yhat3,yhat4,yhat5,yhat6,yhat7,yhat8,...,ar287,ar288,trend,season_weekly,season_daily,future_regressors_additive,future_regressor_Incep,future_regressor_Tncep,future_regressor_lab_day,future_regressor_lab_hour
383,2023-11-03 23:45:00,0.528188,1.557030,1.592985,1.375498,1.075795,1.163935,0.874681,1.104603,1.089235,...,-0.099387,-0.020490,3.057180,-1.097872,-1.337974,0.729922,0.0,0.729922,0.0,0.0
384,2023-11-04 00:00:00,0.529314,0.474293,1.479069,1.509887,1.313222,1.032888,1.130510,0.869249,1.063605,...,-0.041093,-0.027094,3.057178,-1.128197,-1.419158,0.715226,0.0,0.715226,0.0,0.0
385,2023-11-04 00:15:00,0.547498,0.494786,0.385513,1.364424,1.403126,1.218266,0.934913,1.034318,0.803671,...,-0.051123,-0.166629,3.057176,-1.152355,-1.474268,0.700529,0.0,0.700529,0.0,0.0
386,2023-11-04 00:30:00,0.532453,0.524480,0.443521,0.329375,1.231127,1.258062,1.080267,0.830220,0.875307,...,-0.031650,0.076883,3.057173,-1.188330,-1.524258,0.687466,0.0,0.687466,0.0,0.0
387,2023-11-04 00:45:00,0.545137,0.478503,0.468277,0.386613,0.252385,1.116016,1.176152,0.987427,0.777086,...,-0.070963,-0.134774,3.057172,-1.212134,-1.539061,0.672769,0.0,0.672769,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5564,2023-12-27 23:00:00,2.412793,2.440763,2.617723,2.717664,2.677439,2.777573,2.434362,2.428489,2.472755,...,0.045349,0.053194,3.046556,1.026839,-0.990003,0.447424,0.0,0.447424,0.0,0.0
5565,2023-12-27 23:15:00,2.247221,2.374541,2.357720,2.552689,2.648023,2.580688,2.695983,2.360294,2.291351,...,0.078370,0.088330,3.046554,1.035329,-1.109267,0.431095,0.0,0.431095,0.0,0.0
5566,2023-12-27 23:30:00,2.984895,2.321478,2.391031,2.396703,2.593157,2.693638,2.677596,2.793784,2.512339,...,0.110373,0.120796,3.046552,1.045465,-1.220429,0.416398,0.0,0.416398,0.0,0.0
5567,2023-12-27 23:45:00,1.627003,2.979979,2.319606,2.391565,2.419564,2.567045,2.664228,2.624058,2.643347,...,0.142251,0.152483,3.046550,1.056051,-1.336151,0.401702,0.0,0.401702,0.0,0.0


In [ ]:
forecast[['ds','y','season_daily','season_weekly','trend']+ [f'yhat{i+1}' for i in range(288)]].to_csv('load_Dayahead_20231103_20231228.csv',index=False)

In [ ]:
#forecast.to_csv('load_Dayahead_20231101_20231228.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np
#from neuralprophet import NeuralProphet
import matplotlib.pyplot as plt
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.graph_objects as go
import plotly.express as px
forecast = pd.read_csv('/content/drive/MyDrive/data for senior prj/load_Dayahead_20231103_20231228.csv',parse_dates=['ds'])
forecast

,ds,y,season_daily,season_weekly,trend,yhat1,yhat2,yhat3,yhat4,yhat5,...,yhat279,yhat280,yhat281,yhat282,yhat283,yhat284,yhat285,yhat286,yhat287,yhat288
0,2023-11-03 23:45:00,0.528188,-1.337974,-1.097872,3.057180,1.557030,1.592985,1.375498,1.075795,1.163935,...,1.341861,1.264289,1.229870,1.255223,1.311029,1.288671,1.251396,1.312752,1.251869,1.330767
1,2023-11-04 00:00:00,0.529314,-1.419158,-1.128197,3.057177,0.474293,1.479069,1.509888,1.313222,1.032888,...,1.035956,1.175201,1.153721,1.145810,1.172560,1.140950,1.226734,1.140983,1.183955,1.197954
2,2023-11-04 00:15:00,0.547498,-1.474268,-1.152355,3.057176,0.494786,0.385512,1.364424,1.403126,1.218266,...,1.108687,1.060483,1.071750,1.037664,0.987085,1.095013,1.010480,1.104904,1.079959,0.964453
3,2023-11-04 00:30:00,0.532453,-1.524258,-1.188330,3.057174,0.524480,0.443521,0.329375,1.231127,1.258063,...,1.040030,0.998213,1.005244,1.026055,1.034690,0.939097,1.029654,0.970434,1.000401,1.108934
4,2023-11-04 00:45:00,0.545137,-1.539061,-1.212134,3.057172,0.478503,0.468277,0.386613,0.252385,1.116016,...,0.833054,0.951609,0.915890,0.929939,0.895676,0.910270,0.843436,0.928523,0.907782,0.843971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5181,2023-12-27 23:00:00,2.412793,-0.990003,1.026839,3.046556,2.440764,2.617723,2.717664,2.677439,2.777573,...,3.599753,3.576401,3.571148,3.568664,3.573096,3.584614,3.585424,3.582229,3.576166,3.584010
5182,2023-12-27 23:15:00,2.247221,-1.109268,1.035329,3.046554,2.374541,2.357720,2.552689,2.648023,2.580688,...,3.502116,3.477622,3.476360,3.474489,3.479515,3.485756,3.489218,3.485244,3.482080,3.492040
5183,2023-12-27 23:30:00,2.984895,-1.220429,1.045465,3.046552,2.321478,2.391031,2.396704,2.593157,2.693638,...,3.417037,3.392667,3.391495,3.390677,3.390804,3.403096,3.404405,3.402562,3.398360,3.408783
5184,2023-12-27 23:45:00,1.627003,-1.336151,1.056051,3.046550,2.979979,2.319606,2.391565,2.419564,2.567045,...,3.331856,3.306105,3.304449,3.305419,3.310730,3.313435,3.319212,3.313561,3.310403,3.320635


## whole error

In [ ]:
error_df = pd.DataFrame()
norm_factor = forecast['y'].max() - forecast['y'].min()
#get e = y_true - y_pred
for i in range(288) :
  error_df[f'e{i+1}'] =  forecast['y'] - forecast[f'yhat{i+1}']

error_df

<ipython-input-6-53005aa8cdb9>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  error_df[f'e{i+1}'] =  forecast['y'] - forecast[f'yhat{i+1}']
<ipython-input-6-53005aa8cdb9>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  error_df[f'e{i+1}'] =  forecast['y'] - forecast[f'yhat{i+1}']
<ipython-input-6-53005aa8cdb9>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,...,e279,e280,e281,e282,e283,e284,e285,e286,e287,e288
0,-1.028842,-1.064797,-0.847311,-0.547607,-0.635748,-0.346493,-0.576416,-0.561048,-0.835785,-1.186283,...,-0.813673,-0.736101,-0.701682,-0.727035,-0.782841,-0.760483,-0.723209,-0.784564,-0.723682,-0.802579
1,0.055021,-0.949755,-0.980573,-0.783908,-0.503574,-0.601196,-0.339934,-0.534291,-0.532935,-0.744822,...,-0.506642,-0.645887,-0.624407,-0.616496,-0.643246,-0.611636,-0.697420,-0.611669,-0.654641,-0.668640
2,0.052712,0.161985,-0.816926,-0.855628,-0.670768,-0.387415,-0.486820,-0.256174,-0.427971,-0.436579,...,-0.561190,-0.512986,-0.524252,-0.490166,-0.439587,-0.547516,-0.462982,-0.557406,-0.532461,-0.416956
3,0.007973,0.088932,0.203079,-0.698674,-0.725609,-0.547814,-0.297767,-0.342854,-0.148900,-0.302259,...,-0.507577,-0.465760,-0.472791,-0.493602,-0.502237,-0.406644,-0.497201,-0.437981,-0.467948,-0.576481
4,0.066634,0.076860,0.158524,0.292752,-0.570878,-0.631015,-0.442289,-0.231949,-0.257966,-0.062469,...,-0.287917,-0.406472,-0.370753,-0.384802,-0.350539,-0.365133,-0.298299,-0.383386,-0.362645,-0.298833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5181,-0.027970,-0.204929,-0.304871,-0.264645,-0.364779,-0.021569,-0.015696,-0.059961,-0.676666,0.016073,...,-1.186959,-1.163608,-1.158355,-1.155871,-1.160303,-1.171821,-1.172631,-1.169436,-1.163372,-1.171217
5182,-0.127320,-0.110499,-0.305468,-0.400802,-0.333467,-0.448762,-0.113073,-0.044130,-0.073817,-0.674631,...,-1.254895,-1.230401,-1.229139,-1.227268,-1.232294,-1.238535,-1.241997,-1.238023,-1.234859,-1.244819
5183,0.663417,0.593864,0.588192,0.391738,0.291257,0.307299,0.191111,0.472556,0.546561,0.565573,...,-0.432141,-0.407772,-0.406600,-0.405782,-0.405909,-0.418201,-0.419510,-0.417667,-0.413465,-0.423888
5184,-1.352976,-0.692603,-0.764562,-0.792561,-0.940042,-1.037225,-0.997055,-1.016344,-0.842813,-0.801226,...,-1.704853,-1.679102,-1.677446,-1.678416,-1.683727,-1.686432,-1.692209,-1.686559,-1.683400,-1.693632


In [ ]:
df = np.abs(error_df.div(forecast['y'],axis=0))
fig = px.histogram(df, x="e96", nbins=1000)
fig.show()

In [ ]:
# MAE
MAE = np.mean(np.abs(error_df),axis=0)
# RMSE
RMSE = np.sqrt( np.mean(error_df**2,axis=0))
# MAPE
MAPE = np.mean(np.abs(error_df.div(forecast['y'],axis=0)), axis=0)*100
# MBE
MBE = np.mean(error_df,axis=0)
# minmax NMAE
NMAE =  np.mean(np.abs(error_df),axis=0)/norm_factor
error_table = pd.DataFrame({'step':range(1,289),'MAE':MAE,'RMSE':RMSE,'MAPE':MAPE,'MBE':MBE,'NMAE':NMAE})



In [ ]:
error_table[(error_table['step'] == 96)| (error_table['step'] == 192) | (error_table['step'] == 288)]

,step,MAE,RMSE,MAPE,MBE,NMAE
e96,96,1.610739,2.080651,119.893563,-0.713551,0.100449
e192,192,1.734993,2.248028,127.553984,-0.873638,0.108198
e288,288,1.767840,2.281310,130.010662,-0.909892,0.110246


##Error by hour

In [ ]:
df_eval = forecast[['ds',    'y'] + [ f'yhat{i+1}' for i in range(288)]]
error_df = pd.DataFrame()
error_df['ds'] = df_eval['ds']
error_df['y'] = df_eval['y']
#get e = y_true - y_pred
for i in range(288) :
  error_df[f'e{i+1}'] =  forecast['y'] - forecast[f'yhat{i+1}']

error_df

<ipython-input-10-9036368f7415>:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

<ipython-input-10-9036368f7415>:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

<ipython-input-10-9036368f7415>:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

<ipython-input-10-9036368f7415>:7: PerformanceWarning:

Da

,ds,y,e1,e2,e3,e4,e5,e6,e7,e8,...,e279,e280,e281,e282,e283,e284,e285,e286,e287,e288
0,2023-11-03 23:45:00,0.528188,-1.028842,-1.064797,-0.847311,-0.547607,-0.635748,-0.346493,-0.576416,-0.561048,...,-0.813673,-0.736101,-0.701682,-0.727035,-0.782841,-0.760483,-0.723209,-0.784564,-0.723682,-0.802579
1,2023-11-04 00:00:00,0.529314,0.055021,-0.949755,-0.980573,-0.783908,-0.503574,-0.601196,-0.339934,-0.534291,...,-0.506642,-0.645887,-0.624407,-0.616496,-0.643246,-0.611636,-0.697420,-0.611669,-0.654641,-0.668640
2,2023-11-04 00:15:00,0.547498,0.052712,0.161985,-0.816926,-0.855628,-0.670768,-0.387415,-0.486820,-0.256174,...,-0.561190,-0.512986,-0.524252,-0.490166,-0.439587,-0.547516,-0.462982,-0.557406,-0.532461,-0.416956
3,2023-11-04 00:30:00,0.532453,0.007973,0.088932,0.203079,-0.698674,-0.725609,-0.547814,-0.297767,-0.342854,...,-0.507577,-0.465760,-0.472791,-0.493602,-0.502237,-0.406644,-0.497201,-0.437981,-0.467948,-0.576481
4,2023-11-04 00:45:00,0.545137,0.066634,0.076860,0.158524,0.292752,-0.570878,-0.631015,-0.442289,-0.231949,...,-0.287917,-0.406472,-0.370753,-0.384802,-0.350539,-0.365133,-0.298299,-0.383386,-0.362645,-0.298833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5181,2023-12-27 23:00:00,2.412793,-0.027970,-0.204929,-0.304871,-0.264645,-0.364779,-0.021569,-0.015696,-0.059961,...,-1.186959,-1.163608,-1.158355,-1.155871,-1.160303,-1.171821,-1.172631,-1.169436,-1.163372,-1.171217
5182,2023-12-27 23:15:00,2.247221,-0.127320,-0.110499,-0.305468,-0.400802,-0.333467,-0.448762,-0.113073,-0.044130,...,-1.254895,-1.230401,-1.229139,-1.227268,-1.232294,-1.238535,-1.241997,-1.238023,-1.234859,-1.244819
5183,2023-12-27 23:30:00,2.984895,0.663417,0.593864,0.588192,0.391738,0.291257,0.307299,0.191111,0.472556,...,-0.432141,-0.407772,-0.406600,-0.405782,-0.405909,-0.418201,-0.419510,-0.417667,-0.413465,-0.423888
5184,2023-12-27 23:45:00,1.627003,-1.352976,-0.692603,-0.764562,-0.792561,-0.940042,-1.037225,-0.997055,-1.016344,...,-1.704853,-1.679102,-1.677446,-1.678416,-1.683727,-1.686432,-1.692209,-1.686559,-1.683400,-1.693632


In [ ]:
K = 288 # number of step ahead
ds_hour = pd.DataFrame({'hour of target':[f'{hour:02}:00' for hour in range(24)]})
# calculate metric by hour
# iterate over hour
MAE = []
RMSE = []
MAPE = []
MBE = []
NMAE = []
SD = []
MEAN = []
for hour in range(24) :
  hour_df = error_df[error_df['ds'].dt.hour == hour][[f'e{k+1}'  for k in range(K)]]
  y = error_df[error_df['ds'].dt.hour == hour]['y']
  SD.append(y.std(ddof=0))
  MEAN.append(y.mean())
  norm_factor = y.max() - y.min()
  # MAE
  MAE.append( np.mean(np.abs(hour_df),axis=0))
  # RMSE
  RMSE.append(np.sqrt( np.mean(hour_df**2,axis=0)) )
  # MAPE
  MAPE.append( np.mean(np.abs(hour_df.div(y,axis=0)), axis=0)*100 )
  # MBE
  MBE.append(np.mean(hour_df,axis=0) )
  # minmax NMAE
  NMAE.append ( np.mean(np.abs(hour_df),axis=0)/norm_factor)
MAE = pd.DataFrame(MAE)
RMSE = pd.DataFrame(RMSE)
MAPE = pd.DataFrame(MAPE)
MBE = pd.DataFrame(MBE)
NMAE = pd.DataFrame(NMAE)
SD = pd.DataFrame({'SD':SD,'MEAN':MEAN})
MAE['hour of target'] = ds_hour
RMSE['hour of target'] = ds_hour
MAPE['hour of target'] = ds_hour
MBE['hour of target'] = ds_hour
NMAE['hour of target'] = ds_hour
SD['hour of target'] = ds_hour




In [ ]:
SD

,SD,MEAN,hour of target
0,1.018130,1.323395,00:00
1,0.937108,1.305359,01:00
2,0.977718,1.285956,02:00
3,0.728495,1.061467,03:00
4,0.622637,0.941618,04:00
5,0.596345,0.884192,05:00
6,0.713376,0.961493,06:00
7,1.147365,1.433316,07:00
8,1.709307,2.348354,08:00
9,2.054995,2.883614,09:00


In [ ]:
!pip install kaleido
import kaleido

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(x=SD['hour of target'], y=SD['MEAN'] ,error_y=dict(type='data',array=SD['SD'])))

fig.update_layout(
    title='Mean and standard deviation of load by hour',
    xaxis_title='Hour',
    yaxis_title= 'Power (kW) ',
    barmode='group',
    width=2480,
    height=1000,
    font = {'size' : 24},
    plot_bgcolor='white',
                  xaxis=dict(
                      linecolor='black',
                      showgrid=True,
                      gridcolor='lightgrey'
                  ),
                  yaxis=dict(
                      linecolor='black',
                      showgrid=True,
                      gridcolor='lightgrey'
                  )
)
#fig.write_image('load_Mean_SD.pdf')
fig.show()

In [ ]:
import plotly.graph_objects as go

# Assuming mae_hour_df is your DataFrame
step = [1,2,3]
fig = go.Figure()
color = ['#000000','#FF0000','#00FF00','#0000FF','#FF9900']
metric = 'MAPE'
for k in step:
    fig.add_trace(go.Bar(x=MAPE['hour of target'], y=MAPE[f'e{k*96}'], name=str(k) +'day ahead'))

fig.update_layout(
    title=metric + ' of predicted load by hour',
    xaxis_title='Hour',
    yaxis_title=metric + ' (%)',
    barmode='group',
    width=2480,
    height=1000,
    font = {'size' : 24},
    plot_bgcolor='white',
                  xaxis=dict(
                      linecolor='black',
                      showgrid=True,
                      gridcolor='black'
                  ),
                  yaxis=dict(
                      linecolor='black',
                      showgrid=True,
                      gridcolor='black'
                  )
)

fig.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.graph_objects as go

# Assuming result_df is your DataFrame

# Create Plotly time series plot
ts_fig = go.Figure()
step = [96,192,288]


ts_fig.add_trace(go.Scatter(x=df_eval['ds'], y=df_eval['y'], name='Actual', mode='lines',line=dict(color='#000000') ) )
for k in step :
  ts_fig.add_trace(go.Scatter(x=df_eval['ds'], y=df_eval[f'yhat{k}'], name=str(k//96) +' day ahead', mode='lines')

                   )


ts_fig.update_layout(title='Predicted and actual load consumption',
                  xaxis_title='Datetime',
                  yaxis_title='Ptot (kW)',
                  width=2480,
                  height=1000,
                  font = {'size' : 24},
                  plot_bgcolor='white',
                  xaxis=dict(
                      linecolor='black',
                      showgrid=True,
                      gridcolor='black'
                  ),
                  yaxis=dict(
                      linecolor='black',
                      showgrid=True,
                      gridcolor='black'
                  )
                     )

ts_fig.show()

In [ ]:
ts_fig = go.Figure()

ts_fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['season_daily'], name='daily_seasonality', mode='lines'))
ts_fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['season_weekly'], name='weekly_seasonality', mode='lines'))
ts_fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['trend'], name='trend', mode='lines'))

ts_fig.update_layout(title='Seasonality and trend component decompose by NeuralProphet',
                  xaxis_title='Datetime',
                  yaxis_title='Power (kW)',
                  width=2480,
                  height=1000,
                  font = {'size' : 24},
                  plot_bgcolor='white',
                  xaxis=dict(
                      linecolor='black',
                      showgrid=True,
                      gridcolor='black'
                  ),
                  yaxis=dict(
                      linecolor='black',
                      showgrid=True,
                      gridcolor='black'
                  )
                     )

ts_fig.show()